# Top Tagging with Particle Flow Network

Documentation: [Examples](https://energyflow.network/examples/), [Architectures](https://energyflow.network/docs/archs/)

In [16]:
# Make tensorflow quieter
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Computing imports
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, roc_curve

# Useful imports
from tqdm import tqdm
import matplotlib.pyplot as plt

# Energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.utils import data_split, to_categorical

In [2]:
# Make sure tensorflow can use the GPU
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# matplotlib settings
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.autolayout'] = True

## Load data

In [5]:
jets_path = "/usatlas/atlas01/atlasdisk/users/atlas_wifeng/phys427/top-tagging/data/jets-988K-padded.npz"
jets = np.load(jets_path)

X, y = jets["X"], jets["y"]
Y = y

print("Loaded data")

Loaded data


### Feature information

`X`: `(N, max_jet_size, 5)` 3D numpy array of floats
* `0`: $p_T$, transverse momentum
* `1`: $\eta$, angular coordinate
* `2`: $\phi$, angular coordinate
* `3`: $E$, energy
* `4`: $R$, $\sqrt{\eta^2 + \phi^2}$

`y`: `(N, 5)` 1D numpy array of ints that are one-hot encoded
* `0`: Gluon
* `1`: Light quark
* `2`: W boson
* `3`: Z boson
* `4`: Top quark


In [8]:
# Do train/val/test split
n_val = 200000
n_test = 200000

(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=n_val, test=n_test)

print("Done train/val/test split")

Done train/val/test split


## Build the model

In [31]:
# Hyperparameters

# Network architecture parameters
Phi_sizes = [100, 100, 256]
F_sizes = [100, 100, 100]

# Network training parameters
n_epochs = 10
batch_size = 500

In [30]:
# Might take a few seconds
pfn = PFN(input_dim=X.shape[-1],
          output_dim=Y.shape[-1],
          Phi_sizes=Phi_sizes,
          F_sizes=F_sizes)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, 5)]    0           []                               
                                                                                                  
 tdist_0 (TimeDistributed)      (None, None, 100)    600         ['input[0][0]']                  
                                                                                                  
 activation_42 (Activation)     (None, None, 100)    0           ['tdist_0[0][0]']                
                                                                                                  
 tdist_1 (TimeDistributed)      (None, None, 100)    10100       ['activation_42[0][0]']          
                                                                                            

In [32]:
# Train the model!
pfn.fit(X_train, Y_train,
        epochs=n_epochs,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        verbose=1)

Epoch 1/10
1176/1176 [==============================] - 25s 19ms/step - loss: 0.8367 - acc: 0.6932 - val_loss: 0.7223 - val_acc: 0.7460
Epoch 2/10
1176/1176 [==============================] - 21s 18ms/step - loss: 0.7086 - acc: 0.7490 - val_loss: 0.6930 - val_acc: 0.7567
Epoch 3/10
1176/1176 [==============================] - 21s 18ms/step - loss: 0.6852 - acc: 0.7586 - val_loss: 0.6680 - val_acc: 0.7659
Epoch 4/10
1176/1176 [==============================] - 21s 18ms/step - loss: 0.6711 - acc: 0.7648 - val_loss: 0.6586 - val_acc: 0.7690
Epoch 5/10
1176/1176 [==============================] - 21s 18ms/step - loss: 0.6618 - acc: 0.7681 - val_loss: 0.6694 - val_acc: 0.7638
Epoch 6/10
1176/1176 [==============================] - 21s 18ms/step - loss: 0.6573 - acc: 0.7692 - val_loss: 0.6495 - val_acc: 0.7725
Epoch 7/10
1176/1176 [==============================] - 21s 18ms/step - loss: 0.6506 - acc: 0.7720 - val_loss: 0.6501 - val_acc: 0.7728
Epoch 8/10
1176/1176 [==========================

## Evaluate model

In [36]:
# get predictions on test data
preds = pfn.predict(X_test, batch_size=500)

400/400 [==============================] - 2s 5ms/step


In [37]:
test_labels = np.argmax(Y_test, axis=1)
pred_labels = np.argmax(preds, axis=1)

In [38]:
mask = (test_labels == pred_labels).astype(float)
print(mask)
print(f"Test accuracy: {mask.mean()}")

[1. 1. 1. ... 1. 1. 0.]
Test accuracy: 0.77748
